In [50]:
import os
import re
import scrapy
import shutil
import scrapy
from bs4 import BeautifulSoup
from scrapy.crawler import CrawlerProcess
import requests
from urllib.parse import unquote
from time import sleep
import pandas as pd


In [15]:
choices = ['https://leagueoflegends.fandom.com/wiki/Category:Artillery_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Assassin_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Battlemage_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Burst_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Catcher_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Controller_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Diver_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Enchanter_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Fighter_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Juggernaut_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Mage_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Marksman_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Skirmisher_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Slayer_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Specialist_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Tank_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Vanguard_champion',
 'https://leagueoflegends.fandom.com/wiki/Category:Warden_champion']
choice1 = choices[0]
choice1


'https://leagueoflegends.fandom.com/wiki/Category:Artillery_champion'

In [48]:
final_answers = {}
for choice in choices:
    category = choice[:-9].split(":")[-1]
    r = requests.get(choice)
    answers = [unquote(x)[5:][:-4] for x in set(re.findall("wiki/[^/]+/LoL", r.text))]
    final_answers[category] = answers
    sleep(3)
final_answers


{'Artillery': ['Ziggs', 'Xerath', 'Lux', 'Varus', "Vel'Koz", 'Jayce'],
 'Assassin': ['Yone',
  'Katarina',
  'Akali',
  "Kha'Zix",
  'Pyke',
  'Fizz',
  'Diana',
  'Rengar',
  'Shaco',
  'Ekko',
  'Kassadin',
  'Qiyana',
  'Zed',
  'Evelynn',
  'Nocturne',
  'Talon'],
 'Battlemage': ['Viktor',
  'Rumble',
  'Anivia',
  'Cassiopeia',
  'Ryze',
  'Swain',
  'Karthus',
  'Taliyah',
  'Vladimir',
  'Aurelion_Sol',
  'Malzahar'],
 'Burst': ['Orianna',
  'Sylas',
  'Lissandra',
  'Syndra',
  'Brand',
  'Veigar',
  'Lux',
  'Zoe',
  'Neeko',
  'Annie',
  'Karma',
  'LeBlanc',
  'Twisted_Fate',
  'Ahri',
  'Seraphine'],
 'Catcher': ['Bard',
  'Pyke',
  'Jhin',
  'Neeko',
  'Blitzcrank',
  'Rakan',
  'Thresh',
  'Ivern',
  'Morgana',
  'Zyra'],
 'Controller': [],
 'Diver': ['Pantheon',
  'Irelia',
  'Camille',
  'Warwick',
  'Hecarim',
  'Olaf',
  'Skarner',
  'Vi',
  "Rek'Sai",
  'Diana',
  'Elise',
  'Rengar',
  'Xin_Zhao',
  'Renekton',
  'Wukong',
  'Jarvan_IV',
  'Lee_Sin'],
 'Enchanter': 

In [54]:
rows

[{'class': 'Artillery', 'champion': 'Ziggs'},
 {'class': 'Artillery', 'champion': 'Xerath'},
 {'class': 'Artillery', 'champion': 'Lux'},
 {'class': 'Artillery', 'champion': 'Varus'},
 {'class': 'Artillery', 'champion': "Vel'Koz"},
 {'class': 'Artillery', 'champion': 'Jayce'},
 {'class': 'Assassin', 'champion': 'Yone'},
 {'class': 'Assassin', 'champion': 'Katarina'},
 {'class': 'Assassin', 'champion': 'Akali'},
 {'class': 'Assassin', 'champion': "Kha'Zix"},
 {'class': 'Assassin', 'champion': 'Pyke'},
 {'class': 'Assassin', 'champion': 'Fizz'},
 {'class': 'Assassin', 'champion': 'Diana'},
 {'class': 'Assassin', 'champion': 'Rengar'},
 {'class': 'Assassin', 'champion': 'Shaco'},
 {'class': 'Assassin', 'champion': 'Ekko'},
 {'class': 'Assassin', 'champion': 'Kassadin'},
 {'class': 'Assassin', 'champion': 'Qiyana'},
 {'class': 'Assassin', 'champion': 'Zed'},
 {'class': 'Assassin', 'champion': 'Evelynn'},
 {'class': 'Assassin', 'champion': 'Nocturne'},
 {'class': 'Assassin', 'champion': 'Tal

In [64]:
my_lookup_ids = {"Artillery": 6,
"Assassin": 11,
"Battlemage": 10,
"Burst": 6,
"Catcher": 8,
"Diver": 2,
"Enchanter": 4,
"Juggernaut": 3,
"Marksman": 5,
"Skirmisher": 9,
"Specialist": 0,
"Vanguard": 1,
"Warden": 7}

redefine_roles_specialists = {"Gnar": 9,
"Azir": 6,
"Fiddlesticks": 2,
"Teemo": 10,
"Heimerdinger": 6,
"Nidalee": 6,
"Quinn": 5,
"Singed": 1,
"Graves": 5,
"Cho'Gath": 7,
"Kayle": 5,
"Zilean": 7,
"Kennen": 6,
"Gangplank": 9,}



classes = []
champions = []
for (x, y) in final_answers.items():
    x = my_lookup_ids.get(x, -1)
    for champion in y:
        if champion in redefine_roles_specialists:
            x = redefine_roles_specialists[champion]
        classes.append(x)
        champions.append(champion)
df = pd.DataFrame()
df['class'] = classes
df['champion'] = champions
df


,class,champion
0,6,Ziggs
1,6,Xerath
2,6,Lux
3,6,Varus
4,6,Vel'Koz
...,...,...
164,7,Poppy
165,7,Shen
166,7,Tahm_Kench
167,7,Galio


In [65]:
df.to_csv("categories.csv", index=False)
